In [1]:
from nnclr import *
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import time 
import scipy
import sklearn

from waveform_data import WaveFormDataset
tf.random.set_seed(0)

C:\ProgramData\Anaconda3\envs\sg\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
train_batch_size = 32 
train_min_snr = 5

train_dataset = WaveFormDataset(
    ["..\\supervised\\data\\hybrid_static_siprobe_64C_600_S11"
     ,"..\\supervised\\data\\REC_32C_600S_31"
    ], 
    batch_size=train_batch_size, 
    min_snr = train_min_snr,
    augmentation = True
)()

train_dataset = dset.create_sg_dataset(["..\\data\\hybrid_static_siprobe_64C_600_S11","..\\data\\REC_32C_600S_31"], noise1=1)


In [ ]:
INITIAL_LR = 0.001
DECAY_STEPS = 3000

temperature = 0.1
queue_size = 10000

LR = tf.keras.experimental.CosineDecay(INITIAL_LR, DECAY_STEPS, 1e-1)


#LR = 0.0001
model = NNCLR(temperature=temperature, queue_size=queue_size)
model.compile(
    contrastive_optimizer=keras.optimizers.Adam(LR),
    probe_optimizer=keras.optimizers.Adam(LR),
)


In [ ]:
model.fit(
    train_dataset, 
    epochs=500
)


In [ ]:
main_res = []

target_samples = []

it = iter(train_dataset)
for i in range(10):
    sample = next(it)
    
    result = model.encoder(sample[0])
        
    result = tf.reshape(result, [-1,width])
    
    target_samples.extend(tf.squeeze(sample[1]))
    
    main_res.extend(result)
    
target_samples = np.array(target_samples)
contrastive_labels = tf.zeros(target_samples.shape[0]) # batch size 

unique_batch_values = np.unique(target_samples, axis=0)

for i in range(unique_batch_values.shape[0]):

    eq = tf.equal(unique_batch_values[i], target_samples)
    
    new_labels = tf.math.reduce_all(eq, axis=-1)
    contrastive_labels = tf.where(new_labels, i, contrastive_labels)


main_labels = np.array(contrastive_labels)
main_res = np.array(main_res)

print(main_res.shape)
print(main_labels.shape)

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
pca = PCA(n_components=2)
tsne = TSNE(n_components=2, learning_rate='auto', init='random', verbose=True, perplexity=30)

print(main_res)

result = pca.fit_transform(main_res)
result = tsne.fit_transform(main_res)

def res(x, label):
    
    uni = np.unique(label)
    for u in uni:
        
        lu = label[label==u]
        xu = x[label==u]
        
        rgb = np.random.rand(3,)
        plt.scatter(xu[:,0], xu[:,1], color=[rgb])
        
        plt.show()
    
res(result, main_labels)

In [ ]:
save_filename = "checkpoints\\ckpt_"+str(int(time.time()))
if False:
    model.encoder.save(save_filename+"\\encoder_high_acc.h5")
    print(save_filename)